In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
%load_ext cython

In [3]:
import ROOT
import h5py
import os
from matplotlib import pyplot as plt
import numpy as np
import time
import sys
import datetime
import time

ROOT.gSystem.Load("$MARSSYS/libmars.so")

0

In [4]:
#filepath_in = "/remote/ceph/user/m/mhuetten/MAGIC_data/sorcerer/"
#filename_M1 = "20180821_M1_05074767.014_Y_CrabNebula-W0.40+035.root"
##filename_M2 = "20180821_M2_05074767.014_D_CrabNebula-W0.40+035.root"

filepath_in  = "/remote/ceph/user/y/yusuke/cta/MAGIC_LST/"
filepath_out = "/remote/ceph/user/m/mhuetten/MAGIC_data/ctapipe/"

# Observation data files
filename_M1 = "20180821_M1_05074760.001_Y_CrabNebula-W0.40+215.root"
filename_M2 = "20180821_M2_05074760.001_Y_CrabNebula-W0.40+215.root"

# Monte Carlo files
#filename_M1 = "GA_M1_za05to35_9_188880_Y_w0.root"
#filename_M2 = "GA_M2_za05to35_9_188880_Y_w0.root"

ntels = 2

camgeom = ROOT.MGeomCamMagicTwo()
#print camgeom.GetPixels().At(4).Print() 

camera = ROOT.MHCamera(camgeom)

filenames = [filename_M1, filename_M2]
filepaths = [os.path.join(filepath_in, filename_M1), os.path.join(filepath_in, filename_M2)]

infile_M1 = ROOT.TFile(os.path.join(filepath_in, filename_M1))
infile_M2 = ROOT.TFile(os.path.join(filepath_in, filename_M2))

infiles = [infile_M1, infile_M2]

filename_out = ''.join(filename_M1.split("_M1"))

In [5]:
ROOT.TBrowser()

<ROOT.TBrowser object ("Browser") at 0x55e3ea20f210>

In [6]:
def get_ROOT_array(file, tree, branch, leaf):
    tfile = ROOT.TFile(file)
    ttree = getattr(tfile, tree)
    hist = ROOT.TH1D("hist","hist",100,-100,100)
    ttree.Project("hist", "%s.%s"%(branch,leaf),"","",100000000000,0)
    treeArr = ttree.GetV1()
    result = np.array(np.frombuffer(treeArr,count=int(hist.GetEntries())), dtype = np.double)
    return result

In [7]:
arr_phe_3D = []
arr_arrival_3D = []
StereoEvtNumber_2D = []
TrigPattern_2D = []
DAQEvtNumber_2D = []
ClockCounter_2D = []
TimeDiff_2D = []

for ntel in range(ntels):
    
    Runheader = ROOT.TChain("RunHeaders")
    Runheader.Add(os.path.join(filepath_in, filenames[ntel]))

    head = ROOT.MRawRunHeader()
    Runheader.SetBranchAddress("MRawRunHeader.",head)

    # extract global information out of first event:
    Runheader.GetEntry(0)
    TelescopeNumber =  head.GetTelescopeNumber()
    nevents = head.GetNumEvents()
    nevents = 1000
    #max_event = Events.GetEntries()
    
    RunNumber = head.GetRunNumber()
    RunType = head.GetRunTypeStr()
    if RunType == "Data":
        is_mc = False
    elif RunType == "Monte Carlo":
        is_mc = True
        Core_x_vec = get_ROOT_array(filepaths[ntel], "Events", "MMcEvt", "fCoreX")
        Core_y_vec = get_ROOT_array(filepaths[ntel], "Events", "MMcEvt", "fCoreY")
        h_first_vec = get_ROOT_array(filepaths[ntel], "Events", "MMcEvt", "fZFirstInteraction")
    else:
        is_mc = False
    
    Events = ROOT.TChain("Events")
    Events.Add(os.path.join(filepath_in, filenames[ntel]))
    
    evtdata_phe = ROOT.MCerPhotEvt()
    evtdata_arrival = ROOT.MArrivalTime()

    evtheader = ROOT.MRawEvtHeader()
    if is_mc == True:
        mcevtheader = ROOT.MMcEvt()
 
    Events.SetBranchAddress("MCerPhotEvt.",evtdata_phe)
    Events.SetBranchAddress("MArrivalTime.",evtdata_arrival)
    Events.SetBranchAddress("MRawEvtHeader.",evtheader)
    if is_mc == True:
        Events.SetBranchAddress("MMcEvt.",mcevtheader)
    
    arr_phe_2D = []
    arr_arrival_2D = []
    arr_triggerid = []

    DAQEvtNumber = []
    ClockCounter = []
    NumTrigLvl2 = []
    StereoEvtNumber = []
    TriggerID = []
    TrigPattern = []
    CalibrationPattern = []
    MacrocellsPattern = []
    TimeDiff = []    
    TrigPatternNames = []
    
    if is_mc == True:
        Energy = [] # TeV
        Core_params = [] # m
        X_max = [] # m
        H_first_int = [] # m
        Shower_angles = []
        
    # valid for runs after 2013-12-13, see MARS MTriggerPatternDecode.cc, ll. 224        
    TrigPatternNames.append('L3 trigger')
    TrigPatternNames.append('Calibration')
    TrigPatternNames.append('LIDAR')
    TrigPatternNames.append('Pedestal')
    TrigPatternNames.append('Pulse injection')
    TrigPatternNames.append('Sumtrigger')
    TrigPatternNames.append('Topo trigger')
    TrigPatternNames.append('L1 trigger')
    
    
    start_time = time.time()

    
    for nevent in range(nevents):

        if nevent >= nevents:
           continue

        sys.stdout.write("\r ... processing event " + str(nevent) + ", " \
                         + "%s >>> Percent Done : %.2f%%" % (datetime.datetime.now(), 100*nevent/float(nevents)))

        Events.GetEvent(nevent)
        
        trigid = evtheader.GetTriggerID()
        
        binarypattern = '\t{0:016b}'.format(trigid&0xff) 
        
        triggpattern = (bool(int(binarypattern[-8])), 
                        bool(int(binarypattern[-1])), 
                        bool(int(binarypattern[-2])), 
                        bool(int(binarypattern[-3])), 
                        bool(int(binarypattern[-4])), 
                        bool(int(binarypattern[-5])), 
                        bool(int(binarypattern[-6])), 
                        bool(int(binarypattern[-7])))
                
        # check mono-character of run:
        #if nevent > 0:
         #   if triggpattern[0] == False:
         #       if StereoEvtNumber[-1] == StereoEvtNumber[-2]:
         #          StereoEvtNumber[-1] = -1
                #else:
                   # print "calib!"
        
        npixels = evtdata_phe.GetNumPixels() + 1
        
        camera.SetCamContent(evtdata_phe)
        camera.SetAllUsed()
        arr_phe_evt = []
        for i_pixel in range(npixels):
            arr_phe_evt.append(camera.GetBinContent(i_pixel + 1))

        camera.SetCamContent(evtdata_arrival)
        camera.SetAllUsed()
        arr_arrival_evt = []
        for i_pixel in range(npixels):
            arr_arrival_evt.append(camera.GetBinContent(i_pixel + 1))
            
        if arr_phe_evt != []:
            arr_phe_2D.append(np.array(arr_phe_evt))
            arr_arrival_2D.append(np.array(arr_arrival_evt))
            
            DAQEvtNumber.append(evtheader.GetDAQEvtNumber())
            ClockCounter.append(evtheader.GetClockCounter())
            NumTrigLvl2.append(evtheader.GetNumTrigLvl2())
            StereoEvtNumber.append(evtheader.GetStereoEvtNumber())
            TriggerID.append(evtheader.GetTriggerID())
            TrigPattern.append(triggpattern)
            CalibrationPattern.append(evtheader.GetCalibrationPattern())
            MacrocellsPattern.append(evtheader.GetMacrocellsPattern())
            TimeDiff.append(evtheader.GetTimeDiff())
            if is_mc == True:
                energy_val = str(mcevtheader.GetEnergyStr())[:-3]
                energy_unit = str(mcevtheader.GetEnergyStr())[-3:]
                if energy_unit == "TeV":
                    Energy.append(float(energy_val))
                elif energy_unit == "GeV":
                    Energy.append(float(energy_val) / 1000.)
                else:
                    raise IOError("Error!")
                Core_params.append((Core_x_vec[nevent] / 100., Core_y_vec[nevent] / 100.)) # meters
                H_first_int.append(h_first_vec[nevent] / 100.) # meters

    StereoEvtNumber_2D.append(np.array(StereoEvtNumber, dtype=np.int32))
    DAQEvtNumber_2D.append(np.array(DAQEvtNumber, dtype=np.int32))
    arr_phe_3D.append(np.array(arr_phe_2D))
    arr_arrival_3D.append(np.array(arr_arrival_2D))
    TrigPattern_2D.append(TrigPattern)
    
    ClockCounter_2D.append(np.array(ClockCounter, dtype=np.int64))
    TimeDiff_2D.append(np.array(TimeDiff))
            
    end_time = time.time()
    elapsed = end_time - start_time
    print "\n File read into python arrays >>> Elapsed time: %.1f s" % elapsed


 ... processing event 999, 2018-09-20 22:21:18.202784 >>> Percent Done : 99.90%
 File read into python arrays >>> Elapsed time: 4.1 s
 ... processing event 999, 2018-09-20 22:21:22.043634 >>> Percent Done : 99.90%
 File read into python arrays >>> Elapsed time: 3.8 s


In [8]:
# Catch data events only and make stereo events match:


M1_mjd = get_ROOT_array(filepaths[0], "Events", "MTime", "fMjd")
M1_milliseconds_since_mjd = get_ROOT_array(filepaths[0], "Events", "MTime", "fTime.fMilliSec")
M1_nanoseconds_since_second = get_ROOT_array(filepaths[0], "Events", "MTime", "fNanoSec")
M1_seconds_since_mjd = M1_milliseconds_since_mjd/1000. + M1_nanoseconds_since_second/1e9
M2_mjd = get_ROOT_array(filepaths[1], "Events", "MTime", "fMjd")
M2_milliseconds_since_mjd = get_ROOT_array(filepaths[1], "Events", "MTime", "fTime.fMilliSec")
M2_nanoseconds_since_second = get_ROOT_array(filepaths[1], "Events", "MTime", "fNanoSec")
M2_seconds_since_mjd = M2_milliseconds_since_mjd/1000. + M2_nanoseconds_since_second/1e9

seconds_since_mjd = [M1_seconds_since_mjd, M2_seconds_since_mjd]
mjd = [M1_mjd, M2_mjd]

arr_phe_3D_data = []
arr_arrival_3D_data = []
StereoEvtNumber_2D_data = []
TrigPattern_2D_data = []
tels_with_data = []
TimeStamp = []


# all data events:
eventstream = np.union1d(StereoEvtNumber_2D[0], StereoEvtNumber_2D[1])
nevents_data = len(eventstream)

# all stereo events:
stereoevents = np.intersect1d(StereoEvtNumber_2D[0], StereoEvtNumber_2D[1])
nstereo = len(stereoevents)
nstereo_check = 0



for event_id in eventstream:

    stereopatterns = np.zeros(2, dtype=np.bool)
    stereotimestamps = np.zeros(4) - 1.
    
    arr_phe_2D_data = np.zeros(shape=(2,npixels))
    arr_arrival_2D_data = np.zeros(shape=(2,npixels))
    
    for ntel in range(ntels):
        
        # search event
        if event_id not in StereoEvtNumber_2D[ntel]:
            nevent = -1
        else:
            nevent = np.searchsorted(StereoEvtNumber_2D[ntel], event_id, side='left')
            
        # sort out remaining calibration runs:
        if TrigPattern_2D[ntel][nevent][0] == False:
            nevent = -1    
    
        if nevent == -1:
            arr_phe_2D_data[ntel] = np.zeros(npixels)
            arr_arrival_2D_data[ntel] = np.zeros(npixels)
        else:
            arr_phe_2D_data[ntel] = arr_phe_3D[ntel][nevent,:]
            arr_arrival_2D_data[ntel] = arr_phe_3D[ntel][nevent,:]
            stereopatterns[ntel] = True
            stereotimestamps[ntel] = mjd[ntel][nevent]
            stereotimestamps[ntel + 2] = seconds_since_mjd[ntel][nevent]
                
    TimeStamp.append((stereotimestamps[0], stereotimestamps[1], stereotimestamps[2], stereotimestamps[3]))
    
    if (stereopatterns == [False, False]).all():
        print "Error"
        tels_with_data.append((False, False))
    elif (stereopatterns == [False, True]).all():
        tels_with_data.append((False, True))
    elif (stereopatterns == [True, False]).all():
        tels_with_data.append((True, False))
    elif (stereopatterns == [True, True]).all():
        tels_with_data.append((True, True))
        
        
    arr_phe_3D_data.append(arr_phe_2D_data)
    arr_arrival_3D_data.append(arr_arrival_2D_data)

    
    
arr_phe_3D_data = np.array(arr_phe_3D_data)
arr_arrival_3D_data = np.array(arr_arrival_3D_data)
print arr_arrival_3D_data.shape

import pandas as pd
print pd.DataFrame(StereoEvtNumber_2D)



Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
(773, 2, 1184)
   0    1    2    3    4    5    6    7    8    9    ...    990   991   992  \
0  794  794  795  795  796  796  798  799  799  800  ...   1615  1615  1616   
1  795  796  796  798  799  800  801  802  803  803  ...   1620  1620  1621   

    993   994   995   996   997   998   999  
0  1617  1618  1619  1619  1620  1621  1621  
1  1623  1624  1625  1625  1626  1627  1627  

[2 rows x 1000 columns]


In [9]:
f = h5py.File(os.path.join(filepath_out,filename_out.split(".root")[0] + ".hdf5"),"w")
dsets = []

dsets.append(f.create_dataset("dl1/event_id", data=eventstream))#, compression_opts=9)

dt = np.dtype([("M1", np.bool), ("M2", np.bool)])
dsets.append(f.create_dataset("trig/tels_with_trigger", (nevents_data,), dt))
dsets[-1][...] = np.array(tels_with_data, dtype = dt)
f["/dl1/tels_with_data"] = h5py.SoftLink('/trig/tels_with_trigger')

dt = np.dtype([("M1_mjd", np.int32), ("M2_mjd", np.int32), ("M1_sec", np.double), ("M2_sec", np.double)])
dsets.append(f.create_dataset("trig/gps_time", (nevents_data,), dt))
dsets[-1][...] = np.array(TimeStamp, dtype = dt)
dsets[-1].attrs['FIELD_0_UNIT'] = "d"
dsets[-1].attrs['FIELD_1_UNIT'] = "d"
dsets[-1].attrs['FIELD_2_UNIT'] = "s"
dsets[-1].attrs['FIELD_3_UNIT'] = "s"


for ntel in range(ntels):
    
    ##########################
    # write data to hdf5 file:
    

    
    dsets.append(f.create_dataset("dl1/tel"+ str(ntel+1) +"/image", data=arr_phe_3D_data[:,ntel,:], compression='gzip'))#, compression_opts=9)
    dsets.append(f.create_dataset("dl1/tel"+ str(ntel+1) +"/peakpos", data=arr_arrival_3D_data[:,ntel,:], compression='gzip'))#, compression_opts=9)

    # Raw-level data with non-data events for cross checks:
    dt = np.dtype([(TrigPatternNames[0], np.bool), (TrigPatternNames[1], np.bool), (TrigPatternNames[2], np.bool), 
                   (TrigPatternNames[3], np.bool), (TrigPatternNames[4], np.bool), (TrigPatternNames[5], np.bool), 
                   (TrigPatternNames[6], np.bool), (TrigPatternNames[7], np.bool)])
    dsets.append(f.create_dataset("r0/tel"+ str(ntel+1) +"/TrigPattern", (nevents,), dt))
    dsets[-1][...] = np.array(TrigPattern_2D[ntel], dtype = dt)
    dsets.append(f.create_dataset("r0/tel"+ str(ntel+1) +"/DAQEvtNumber", data=DAQEvtNumber_2D[ntel]))
    dsets.append(f.create_dataset("r0/tel"+ str(ntel+1) +"/StereoEvtNumber", data=StereoEvtNumber_2D[ntel]))
    dsets.append(f.create_dataset("r0/tel"+ str(ntel+1) +"/ClockCounter", data=ClockCounter_2D[ntel]))
    dsets.append(f.create_dataset("r0/tel"+ str(ntel+1) +"/TimeDiff", data=TimeDiff_2D[ntel]))
    
    # write MC information:
    if is_mc == True and ntel == 1:
        dsets_mcheader = []
        
        dt = np.dtype([("Energy", np.double),])
        dsets_mcheader.append(f.create_dataset("McHeader/Energy", (nevents_check,), dt))
        dsets_mcheader[-1][...] = np.array(Energy, dtype = dt)
        dsets_mcheader[-1].attrs['FIELD_0_NAME'] = "Energy"
        dsets_mcheader[-1].attrs['FIELD_0_UNIT'] = "TeV"
        
        dt = np.dtype([("Core_x", np.double), ("Core_y", np.double)])
        dsets_mcheader.append(f.create_dataset("McHeader/Core_xy", (nevents_check,), dt))
        dsets_mcheader[-1][...] = np.array(Core_params, dtype = dt)
        dsets_mcheader[-1].attrs['FIELD_0_NAME'] = "Core_x"
        dsets_mcheader[-1].attrs['FIELD_1_NAME'] = "Core_y"
        dsets_mcheader[-1].attrs['FIELD_0_UNIT'] = "m"

        dt = np.dtype([("H_first_int", np.double),])
        dsets_mcheader.append(f.create_dataset("McHeader/H_first_int", (nevents_check,), dt))
        dsets_mcheader[-1][...] = np.array(H_first_int, dtype = dt)
        dsets_mcheader[-1].attrs['FIELD_0_NAME'] = "H_first_int"
        dsets_mcheader[-1].attrs['FIELD_0_UNIT'] = "m"
        

f.close()

In [10]:
f.close()